<a href="https://colab.research.google.com/github/prithvivinodnair/Data-Visualization/blob/main/HW3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import plot

# Load dataset
df = pd.read_csv("DV.csv")
df = df.dropna(subset=['iyear', 'attacktype1_txt'])

# Group and count
grouped = df.groupby(['iyear', 'attacktype1_txt']).size().reset_index(name='count')

# Pivot for stacked chart
pivot_df = grouped.pivot(index='iyear', columns='attacktype1_txt', values='count').fillna(0)
pivot_df.reset_index(inplace=True)
pivot_df['total'] = pivot_df.drop(columns=['iyear']).sum(axis=1)

# Normalize for 100% stacked
normalized_df = pivot_df.copy()
for col in normalized_df.columns[1:-1]:
    normalized_df[col] = normalized_df[col] / normalized_df['total']

attack_types = pivot_df.columns[1:-1]

# Build figure
fig = go.Figure()

for attack in attack_types:
    fig.add_trace(go.Scatter(
        x=pivot_df['iyear'],
        y=pivot_df[attack],
        mode='lines',
        stackgroup='one',
        name=attack,
        hoverinfo='x+y+name'
    ))

# Buttons for view toggling
fig.update_layout(
    title="Attack Types Over Time",
    xaxis_title="Year",
    yaxis_title="Number of Incidents",
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.5,
            xanchor="center",
            y=1.15,
            yanchor="top",
            buttons=[
                dict(label="Stacked",
                     method="update",
                     args=[{"y": [pivot_df[attack] for attack in attack_types],
                            "stackgroup": ['one'] * len(attack_types)},
                           {"yaxis": {"title": "Number of Incidents", "tickformat": ""}}]),
                dict(label="Grouped",
                     method="update",
                     args=[{"y": [pivot_df[attack] for attack in attack_types],
                            "stackgroup": [None] * len(attack_types)},
                           {"yaxis": {"title": "Number of Incidents", "tickformat": ""}}]),
                dict(label="100% Stacked",
                     method="update",
                     args=[{"y": [normalized_df[attack] for attack in attack_types],
                            "stackgroup": ['one'] * len(attack_types)},
                           {"yaxis": {"title": "Percentage of Total", "tickformat": ".0%"}}])
            ]
        )
    ]
)

# Save HTML file
plot(fig, filename='attack_types_area_chart.html', auto_open=False)

print(" HTML file created. You can now download 'attack_types_area_chart.html' from the left panel.")


 HTML file created. You can now download 'attack_types_area_chart.html' from the left panel.
